In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'resource'

In [ ]:
mnist_data, info = tfds.load(name="mnist", batch_size=-1, with_info=True)
mnist_data = tfds.as_numpy(mnist_data)
train_data, test_data = mnist_data['train'], mnist_data['test']
X_train, y_train = train_data['image'], train_data['label']
X_test, y_test = test_data['image'], test_data['label']

In [ ]:
X_train = X_train * (1.0/256)
X_test = X_test * (1.0/256)

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
INPUT_SHAPE = (28, 28, 1)
KERNEL_SIZE = (3, 3)
model = Sequential()

# Convolutional Layer
model.add(Conv2D(filters=16, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=16, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
# Pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))
# Dropout layers
model.add(Dropout(0.25))


model.add(Flatten())
# model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=METRICS)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
X_train.shape[0]

In [ ]:
batch_size = 32

data_generator = ImageDataGenerator()
train_generator = data_generator.flow(X_train, y_train, batch_size)
steps_per_epoch = X_train.shape[0] // batch_size

fitted_model = model.fit(train_generator, epochs=10,
              steps_per_epoch=steps_per_epoch,
              validation_data=(X_test, y_test),
               callbacks=[early_stop],
               batch_size=batch_size,
             )